## Import Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import retro 

from skimage import transform 
from skimage.color import rgb2gray 

from collections import deque 

import random 
import warnings

warnings.filterwarnings('ignore')

/home/raed/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Init Environment

In [2]:
env = retro.make(game='SpaceInvaders-Atari2600')

In [3]:
frame_size = env.observation_space 
num_actions = env.action_space.n

print ("Frame Size: \t", frame_size)
print("Action Size: \t", num_actions)

#one hot encoded actions matrix 
actions_1_hot = np.array(np.identity(num_actions, dtype=int).tolist())
print(actions_1_hot)

Frame Size: 	 Box(210, 160, 3)
Action Size: 	 8
[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]]


## Preprocessing 

In [24]:
def preprocess_frame(frame):
    """
    Convert frames to grayscale, crop irrelevent parts, normalize pixels, 
    resize frame.
    
    Input is frame of size 210x160x3
    Returns frame of size 110x84x1 
    """
    grayed = rgb2gray(frame)
    cropped = grayed[8:-12, 4:-12]
    normed = cropped/255.0 
    resized = transform.resize(normed, [110,84])
    
    return resized #returns preprocessed frame 

stack_size = 4
def stack_frames(stacked_frames, frame, is_new_episode, stack_size=stack_size):
    """
    takes a frame/state and preprocesses it, 
    if same episode:
        adds to the stack state 
    else new episode: 
        creates stacked state 
    
    where the stacked state is 4 stacked states(frames)
    returns stacked state where axis=1 is for different frames 
    """
    frame = preprocess_frame(frame)
    if not is_new_episode:
        stacked_frames.append(frame)
        stacked_state = np.stack(stacked_frames, axis=2)
    
    else: 
        #init deque 
        stacked_frames = deque([np.zeros((110,84), dtype=np.int) 
                                for i in range(stack_size)], maxlen=4)
        #new episode so same frame x4 
        for i in range(4): stacked_frames.append(frame)
        
        stacked_state = np.stack(stacked_frames, axis=2)

    return stacked_state, stacked_frames
  

In [25]:
#init frame stack
stacked_frames  =  deque([np.zeros((110,84), dtype=np.int) for i in range(stack_size)], maxlen=4)
frame = np.zeros((110,84), dtype=np.int)

#test helper functions 
sample_state, stacked_frames = stack_frames(stacked_frames, frame, True)
print(np.shape(sample_state), np.shape(stacked_frames))

(110, 84, 4) (4, 110, 84)


## Hyper Params

In [6]:
#env params 
total_episodes = 50 #total num of episodes 
total_test_episodes = 10 #total num of episodes to test on 
max_steps = 50000 #max num of steps per episode 
bs = 64

#network params 
lr = 0.00025
state_size = [110,84,4]

#discount factor 
gamma = 0.9

#exploration params 
epsilon = 1.0 #starting value for eps greedy (explore)
max_eps = 1.0  #max value for eps greey 
min_eps = 0.01 #min value for eps greedy 
decay = 0.00001 #decay rate for eps 

#Recall Params
memory_size = 1000000
pretrained_len = bs #number of init memories 

training=False

## DQN: Model 


In [7]:
class DQN():
    def __init__(self, state_size, action_size, learning_rate, name='DQN', training=training):
        self.state_size = state_size
        self.action_size = action_size
        self.lr = learning_rate
        self.training = training
        with tf.variable_scope(name):
            #placeholders
            self.inputs = tf.placeholder(tf.float32,[None, *self.state_size])
            self.actions = tf.placeholder(tf.float32, [None, self.action_size])
            self.target_Q = tf.placeholder(tf.float32)
            
            #block 1 
            self.conv1 = tf.layers.conv2d(inputs=self.inputs, filters=32, 
                                          kernel_size=[8,8], 
                                          strides=(4,4), 
                                          padding='valid', 
                                          kernel_initializer= tf.contrib.layers.xavier_initializer_conv2d())
            self.bn1 = tf.layers.batch_normalization(self.conv1, epsilon=1e-5, training=self.training)
            self.elu1 = tf.nn.elu(self.bn1)
            
            #block 2
            self.conv2 = tf.layers.conv2d(inputs=self.elu1, filters=64, 
                                          kernel_size=[4,4], 
                                          strides=(2,2), 
                                          padding='valid', 
                                          kernel_initializer= tf.contrib.layers.xavier_initializer_conv2d())
            self.bn2 = tf.layers.batch_normalization(self.conv2, epsilon=1e-5, training=self.training)
            self.elu2 = tf.nn.elu(self.bn2)
            
            #block 3
            self.conv3 = tf.layers.conv2d(inputs=self.elu2, filters=128, 
                                          kernel_size=[4,4], 
                                          strides=(2,2), 
                                          padding='valid', 
                                          kernel_initializer= tf.contrib.layers.xavier_initializer_conv2d())
            self.bn3 = tf.layers.batch_normalization(self.conv3, epsilon=1e-5, training=self.training)
            self.elu3 = tf.nn.elu(self.bn3)
            
            #FC Block 
            self.flat = tf.layers.flatten(self.elu3)
            self.fc = tf.layers.dense(self.flat, units=512, activation=tf.nn.elu, 
                                       kernel_initializer=tf.contrib.layers.xavier_initializer())
            self.out = tf.layers.dense(self.fc, units=self.action_size, activation=None, 
                                       kernel_initializer=tf.contrib.layers.xavier_initializer())
            
            #Q-Value prediction 
            self.pred_Q = tf.reduce_sum(tf.multiply(self.out, self.actions), axis=1)
            
            #Loss Function 
            self.loss = tf.reduce_mean(tf.square(self.target_Q-self.pred_Q))
            
            #Optimizer 
            self.optim = tf.train.RMSPropOptimizer(learning_rate=self.lr).minimize(self.loss)
           

In [14]:
tf.reset_default_graph()

#init 
DQNetwork = DQN(state_size, num_actions, lr)

## Experience Replay

In [15]:
#memory class creates and manages deque 

class Memory():
    def __init__(self, mem_limit):
        self.cache = deque(maxlen=mem_limit)
    
    def add(self, exp):
        self.cache.append(exp)
    
    def sample(self, sample_size):
        sampling_ind = np.random.choice(np.arange(len(self.cache)), size=sample_size, replace=False)
        return [self.cache[i] for i in sampling_ind]

In [16]:
#init memory for first time 
memory = Memory(mem_limit=memory_size)

import pdb 

for i in range(pretrained_len):
    #pdb.set_trace()
    #initilize state for first step  
    if i == 0: 
        state = env.reset() #init state 
        state, stacked_frames = stack_frames(stacked_frames, state, True)
    
    #take a random action, record observations and add if not dead 
    choice = np.random.choice(np.arange(num_actions),size=1, replace=False)
    action = actions_1_hot[choice][0]    
    obs, reward, done, _ = env.step(action)
    
    #comment out if you want unseen 
    env.render()
    
    #stack new frame 
    next_state, stacked_frames = stack_frames(stacked_frames, obs, False)
    
    if not done:
        #add memory 
        memory.add((state, action, reward, next_state, done))
        #update state
        state = next_state
        
    else: #were dead
        #update next state 
        next_state = np.zeros_like(state)
        
        #add exp 
        memory.add((state, action, reward, next_state, done))

        #new episode and restart 
        env.reset()
        state, stacked_frames = stack_frames(stacked_frames, state, True)


## DQN: Training

In [17]:
#Function to predict action based on epsilon greedy 

def explore_exploit(epsilon, min_eps, decay_rate, decay_step, state, num_actions):
    #produce random number between 0 and 1 
    check = np.random.rand()
    explore_prob = min_eps + (epsilon-min_eps)*np.exp(-decay_rate*decay_step)
    
    #explore
    if explore_prob>check:
        #take random action 
        choice = random.randint(1,len(actions_1_hot))-1
        action = actions_1_hot[choice]  
        
    #exploit 
    else:
        #q vals predicted by network 
        Qs = sess.run(DQNetwork.out, feed_dict = {DQNetwork.inputs : state.reshape((1,*state.shape))})
        #choose action corresponding to best Q value 
        choice = np.argmax(Qs)
        action = actions_1_hot[choice]
    return action, explore_prob


In [18]:
#for saving 
saver = tf.train.Saver()
training = False 
episode_render = True 
decay_rate = decay

In [19]:
#training loop
if training == True: 
    with tf.Session() as sess:
        #init vars and decay rate
        sess.run(tf.global_variables_initializer())
        decay_step = 0 
        
        #rewards for each episode 
        episodic_rewards = [] 
        
        for episode in range(total_episodes):
            #init steps for episode 
            step = 0 
            
            #init observation
            state = env.reset()
            state, stacked_frame = stack_frames(stacked_frames, state, True)
            
            #list for rewards collected in episode 
            rewards_list = []
            
            while step < max_steps:
                ################SAMPLING#######################    
                #increments 
                step += 1
                decay_step += 1
                #if step % 50 == 0: print (step)
                #choose action 
                action, explore_prob = explore_exploit(epsilon, min_eps, decay_rate, decay_step, state, num_actions)

                #take action                 
                next_state, reward, done, _ = env.step(action)    
                env.render()
                
                rewards_list.append(reward) 
                
                #if were not dead 
                if not done:
                    #store transition after converting to proper format 
                    next_state, stacked_frames = stack_frames(stacked_frames, obs, False)

                    #add memory 
                    memory.add((state, action, reward, next_state, done))
                    #update state
                    state = next_state

                #if were dead     
                else: 
                    #update next state 
                    next_state = np.zeros_like(state)
                    next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
                    
                    #add episodic rewards 
                    tot_reward = np.sum(rewards_list)
                    episodic_rewards.append((episode, tot_reward))
                    
                    #print episode summary 
                    print(  'Episode: {}'.format(episode),
                            'Total reward: {}'.format(tot_reward),
                            'Explore P: {:.4f}'.format(explore_prob),
                            'Training Loss {:.4f}'.format(loss))
                    
                    #end episode 
                    step = max_steps
                    
                    #add exp 
                    memory.add((state, action, reward, next_state, done))
        
                ################Learning#######################    
                #extract minibatch values 
                mini_batch = memory.sample(bs)
                states_batch = [sample[0] for sample in mini_batch]
                actions_batch = [sample[1] for sample in mini_batch]
                rewards_batch = [sample[2] for sample in mini_batch]
                next_states_batch = [sample[3] for sample in mini_batch]
                dones_batch = [sample[4] for sample in mini_batch]
                
                #target qs to be set below 
                targ_qs_batch = []
                
                #get predicted Q's for each next state 
                next_qs_batch = sess.run(DQNetwork.out, feed_dict={DQNetwork.inputs : next_states_batch})
                
                #determine if state is terminal and set value for target_q
                for i in range(bs):
                    final_state = dones_batch[i]
                    
                    if final_state:
                        targ_qs_batch.append(rewards_batch[i])
                    
                    else:   
                        target_q = rewards_batch[i] + gamma * np.max(next_qs_batch[i])
                        targ_qs_batch.append(target_q)
                    
                
                #convert to np array 
                target_q = np.array([i for i in targ_qs_batch])
                
                #determine loss 
                loss, _ = sess.run([DQNetwork.loss, DQNetwork.optim], feed_dict={DQNetwork.inputs: states_batch,
                                                                               DQNetwork.target_Q: targ_qs_batch,
                                                                               DQNetwork.actions: actions_batch})
                
                
                #perform gradient update 
                
            if episode % 5 == 0:
                save_path = saver.save(sess, "./models/model_{}.ckpt".format(episode))
                print("Model Saved")              

In [29]:

test_episodes = 1 
saver = tf.train.Saver()

with tf.Session() as sess:

    #load the model 
    saver.restore(sess, "./models/model_40.ckpt")
    episodic_rewards = []

    #init state
    state = env.reset()
    state, stacked_frames = stack_frames(stacked_frames, state, True)
    
    #run for each episode  
    for episode in range(test_episodes):
        
        #reward list for single episodes 
        total_rewards = []

        #continue untill the agent dies 
        while True: 
            state = np.reshape(state, (1, *state.shape))
            q_preds = sess.run(DQNetwork.out, feed_dict={DQNetwork.inputs:state})
            choice = np.argmax(q_preds)
            action = actions_1_hot[choice]
            print(q_preds[0])
            #take action 
            new_state, reward, done, _ = env.step(action)
            env.render()

            total_rewards += [reward]
            
            if done:
                print ("""
                >>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
                Reward: {}""".format(np.sum(total_rewards)))
                break 
            
            state, stacked_frames = stack_frames(stacked_frames, new_state, False)

    #env.close()

INFO:tensorflow:Restoring parameters from ./models/model_40.ckpt
[[4.457136  4.4597373 4.4845476 4.496893  4.491561  4.6021643 4.528149
  4.7433043]]
[[4.457111  4.4598904 4.4846115 4.496983  4.4915977 4.602172  4.528305
  4.7433834]]
[[4.4570794 4.460055  4.4846807 4.4970818 4.4916334 4.6021767 4.5284758
  4.7434664]]
[[4.4570537 4.4602118 4.484747  4.4971747 4.4916706 4.602183  4.5286384
  4.7435484]]
[[4.4570155 4.4603767 4.4848175 4.4972744 4.4917045 4.6021843 4.528808
  4.7436314]]
[[4.4570155 4.4603767 4.4848175 4.4972744 4.4917045 4.6021843 4.528808
  4.7436314]]
[[4.4570155 4.4603767 4.4848175 4.4972744 4.4917045 4.6021843 4.528808
  4.7436314]]
[[4.4570155 4.4603767 4.4848175 4.4972744 4.4917045 4.6021843 4.528808
  4.7436314]]
[[4.4570155 4.4603767 4.4848175 4.4972744 4.4917045 4.6021843 4.528808
  4.7436314]]
[[4.457045  4.460224  4.4847507 4.4971824 4.491668  4.6021776 4.52865
  4.7435513]]
[[4.4570727 4.4600596 4.484684  4.497083  4.4916325 4.6021724 4.528481
  4.7434673]]

[[4.4570794 4.460055  4.4846807 4.4970818 4.4916334 4.6021767 4.5284758
  4.7434664]]
[[4.4570537 4.4602118 4.484747  4.4971747 4.4916706 4.602183  4.5286384
  4.7435484]]
[[4.4570155 4.4603767 4.4848175 4.4972744 4.4917045 4.6021843 4.528808
  4.7436314]]
[[4.4570155 4.4603767 4.4848175 4.4972744 4.4917045 4.6021843 4.528808
  4.7436314]]
[[4.4570155 4.4603767 4.4848175 4.4972744 4.4917045 4.6021843 4.528808
  4.7436314]]
[[4.4570155 4.4603767 4.4848175 4.4972744 4.4917045 4.6021843 4.528808
  4.7436314]]
[[4.4570155 4.4603767 4.4848175 4.4972744 4.4917045 4.6021843 4.528808
  4.7436314]]
[[4.457045  4.460224  4.4847507 4.4971824 4.491668  4.6021776 4.52865
  4.7435513]]
[[4.4570727 4.4600596 4.484684  4.497083  4.4916325 4.6021724 4.528481
  4.7434673]]
[[4.457101  4.4599013 4.484616  4.4969907 4.491596  4.6021676 4.528317
  4.743387 ]]
[[4.457136  4.4597373 4.4845476 4.496893  4.491561  4.6021643 4.528149
  4.7433043]]
[[4.457136  4.4597373 4.4845476 4.496893  4.491561  4.6021643 4.

[[4.4125934 4.417226  4.4449344 4.4708667 4.4534197 4.55517   4.5009456
  4.704999 ]]
[[4.412662  4.417288  4.4449935 4.4709573 4.453474  4.5552382 4.500951
  4.7050705]]
[[4.4127297 4.4173393 4.4450507 4.4710326 4.4535213 4.555302  4.500948
  4.7051325]]
[[4.4127965 4.417406  4.4451113 4.4711437 4.4535623 4.5553656 4.5009327
  4.7052107]]
[[4.412858  4.41748   4.445173  4.4712596 4.4535995 4.5554247 4.5009212
  4.70529  ]]
[[4.412925  4.417572  4.4452405 4.471405  4.453644  4.555491  4.5009127
  4.705385 ]]
[[4.4129844 4.4176674 4.445307  4.4715505 4.4536796 4.5555477 4.5009055
  4.7054777]]
[[4.413029  4.4177213 4.4453464 4.4716516 4.4537015 4.5555854 4.500885
  4.7055387]]
[[4.412737  4.419492  4.4468246 4.4729595 4.45338   4.5548344 4.5030627
  4.7061405]]
[[4.4121466 4.4210014 4.4480295 4.4739394 4.4527164 4.553814  4.505028
  4.706457 ]]
[[4.411802  4.422579  4.449311  4.4750338 4.4522514 4.5529695 4.507058
  4.7069016]]
[[4.4111943 4.424114  4.450513  4.4759827 4.4516044 4.55190

[[4.3901887 4.43909   4.4577417 4.4733458 4.4378724 4.5416617 4.52041
  4.7068043]]
[[4.385714  4.439019  4.4569936 4.471333  4.4347486 4.5389085 4.519876
  4.7053757]]
[[4.3857503 4.4390254 4.457004  4.471351  4.4347568 4.5389304 4.5198817
  4.705386 ]]
[[4.3857536 4.439019  4.4570017 4.4713445 4.4347553 4.5389366 4.5198684
  4.705385 ]]
[[4.385938  4.439188  4.457169  4.471512  4.4348507 4.5390515 4.519982
  4.705537 ]]
[[4.3859534 4.439194  4.45718   4.4715185 4.4348516 4.53906   4.519979
  4.7055454]]
[[4.3861775 4.439437  4.457399  4.4717336 4.4349627 4.5391755 4.520165
  4.7057486]]
[[4.3861685 4.4394407 4.4574018 4.4717283 4.434952  4.539164  4.520167
  4.7057486]]
[[4.385984  4.4393635 4.457286  4.47161   4.4348493 4.539007  4.520163
  4.705641 ]]
[[4.385974  4.43937   4.4572897 4.471613  4.434843  4.538996  4.520168
  4.7056417]]
[[4.3859344 4.439334  4.4572525 4.471595  4.4348326 4.5389733 4.520163
  4.7056065]]
[[4.3859353 4.4393306 4.4572554 4.471597  4.434831  4.5389743 4.

[[4.4116936 4.467836  4.4853683 4.48497   4.457993  4.569019  4.535711
  4.731763 ]]
[[4.411693  4.4678264 4.4853587 4.484978  4.4579887 4.5690084 4.535705
  4.7317553]]
[[4.411693  4.4678326 4.485358  4.4849825 4.45799   4.569008  4.5357084
  4.731756 ]]
[[4.4117227 4.467849  4.4853787 4.4850225 4.458007  4.569019  4.535724
  4.7317734]]
[[4.4117255 4.4678545 4.4853816 4.485029  4.45801   4.569019  4.535729
  4.7317753]]
[[4.414697  4.4692    4.487111  4.4862785 4.4603953 4.572269  4.536142
  4.733466 ]]
[[4.417803  4.470582  4.4889174 4.487627  4.4629436 4.575668  4.536557
  4.7352505]]
[[4.420706  4.471901  4.490647  4.488854  4.4653316 4.5789075 4.536921
  4.736952 ]]
[[4.423945  4.473386  4.4925203 4.4901843 4.4679427 4.582481  4.5372944
  4.7388353]]
[[4.4238944 4.473334  4.492483  4.4901023 4.4679036 4.58246   4.5372467
  4.738796 ]]
[[4.4239016 4.473338  4.4924836 4.4901133 4.4679055 4.582459  4.53725
  4.7388   ]]
[[4.423859  4.4732995 4.492453  4.4900594 4.467874  4.58244   4

...

[[4.42174   4.4758887 4.4937325 4.492872  4.468352  4.5835934 4.536365
  4.741433 ]]
[[4.421751  4.475893  4.493735  4.4928875 4.4683547 4.583594  4.536368
  4.741437 ]]
[[4.421743  4.475893  4.4937363 4.492875  4.4683557 4.5835958 4.53637
  4.741438 ]]
[[4.4217377 4.475893  4.4937377 4.492866  4.468357  4.5835986 4.536371
  4.741436 ]]
[[4.421733  4.475892  4.4937367 4.492852  4.468356  4.5835986 4.5363703
  4.741435 ]]
[[4.4217286 4.4758883 4.493735  4.4928365 4.4683547 4.5835986 4.53637
  4.741432 ]]
[[4.4217315 4.4758883 4.4937334 4.492849  4.468353  4.5835958 4.536367
  4.741432 ]]
[[4.4217324 4.475888  4.4937315 4.4928617 4.468352  4.583593  4.536366
  4.741432 ]]
[[4.42174   4.4758887 4.4937325 4.492872  4.468352  4.5835934 4.536365
  4.741433 ]]
[[4.421751  4.475893  4.493735  4.4928875 4.4683547 4.583594  4.536368
  4.741437 ]]
[[4.421743  4.475893  4.4937363 4.492875  4.4683557 4.5835958 4.53637
  4.741438 ]]
[[4.4217377 4.475893  4.4937377 4.492866  4.468357  4.5835986 4.536

[[4.417215  4.4574275 4.4790163 4.481195  4.4644647 4.577855  4.5215974
  4.7312155]]
[[4.4173217 4.4574018 4.4790487 4.4812145 4.46451   4.5779305 4.5215535
  4.7312365]]
[[4.417422  4.457365  4.479056  4.481213  4.464554  4.577988  4.5215254
  4.731242 ]]
[[4.417454  4.4574003 4.4790916 4.481255  4.464581  4.578015  4.521553
  4.7312756]]
[[4.4173627 4.4573402 4.479023  4.4811625 4.464525  4.577954  4.5215216
  4.731203 ]]
[[4.4173465 4.4573927 4.479047  4.4811954 4.4645357 4.577945  4.5215783
  4.73123  ]]
[[4.4172206 4.457358  4.478978  4.4811234 4.464468  4.577859  4.5215716
  4.731165 ]]
[[4.417189  4.4573936 4.4789934 4.4811263 4.464472  4.5778537 4.5216174
  4.7311754]]
[[4.4171443 4.457433  4.478981  4.4811654 4.4644628 4.577811  4.5216618
  4.7311745]]
[[4.4170976 4.457458  4.4789815 4.4811473 4.4644575 4.577794  4.521703
  4.7311716]]
[[4.4171033 4.4575396 4.479005  4.4812436 4.4644794 4.577787  4.521774
  4.731208 ]]
[[4.417049  4.4575624 4.479     4.4812274 4.464465  4.577

[[4.4394526 4.4306154 4.4587326 4.478623  4.4760156 4.587019  4.5066047
  4.7216244]]
[[4.439471  4.4306145 4.4587483 4.4786153 4.4760323 4.587028  4.5066266
  4.721627 ]]
[[4.4392366 4.43039   4.4585247 4.4783225 4.4758873 4.586822  4.506554
  4.7214003]]
[[4.439263  4.430416  4.458554  4.4783497 4.475913  4.586843  4.506576
  4.721427 ]]
[[4.4390483 4.4302135 4.4583435 4.4780974 4.4757824 4.586657  4.5064936
  4.7212276]]
[[4.439084  4.430254  4.4583755 4.4781494 4.4758143 4.586683  4.506519
  4.721265 ]]
[[4.4390597 4.4302363 4.458366  4.4781356 4.475795  4.5866756 4.506482
  4.7212563]]
[[4.4391456 4.4303403 4.458446  4.4782815 4.4758673 4.58674   4.5065365
  4.7213497]]
[[4.4391456 4.4303656 4.458464  4.4783263 4.4758615 4.5867496 4.506518
  4.7213726]]
[[4.439282  4.4305167 4.4585905 4.4785404 4.475972  4.586848  4.506602
  4.72151  ]]
[[4.4393573 4.430627  4.458681  4.4787097 4.47602   4.586905  4.5066347
  4.7216053]]
[[4.439487  4.4307785 4.4588113 4.47892   4.4761305 4.587005

[[4.390997  4.407791  4.4315267 4.457498  4.4378104 4.5439067 4.495088
  4.690859 ]]
[[4.39101   4.407792  4.431528  4.457518  4.437811  4.543905  4.495086
  4.6908607]]
[[4.391013  4.4077973 4.4315333 4.4575295 4.4378138 4.5439067 4.495088
  4.6908655]]
[[4.391027  4.4078    4.431538  4.457548  4.4378176 4.5439067 4.4950905
  4.6908703]]
[[4.3910394 4.4078064 4.4315424 4.457569  4.437821  4.5439076 4.495094
  4.690873 ]]
[[4.3910336 4.4078045 4.4315453 4.4575567 4.4378223 4.5439095 4.4950943
  4.690875 ]]
[[4.3910336 4.4078054 4.431547  4.45755   4.437825  4.5439124 4.495097
  4.690875 ]]
[[4.3910213 4.4078007 4.431545  4.457533  4.4378223 4.5439115 4.4950943
  4.690873 ]]
[[4.3910136 4.4077997 4.4315405 4.457523  4.4378204 4.543911  4.495094
  4.6908693]]
[[4.3910203 4.407798  4.4315405 4.4575343 4.4378176 4.5439076 4.49509
  4.6908684]]
[[4.3910284 4.407803  4.4315457 4.4575486 4.437822  4.5439105 4.495094
  4.690872 ]]
[[4.3910346 4.4078026 4.431545  4.4575605 4.4378204 4.5439076 4

[[4.391128  4.4078546 4.4316287 4.4576716 4.4378824 4.543944  4.49514
  4.69094  ]]
[[4.3911366 4.407859  4.4316306 4.457686  4.4378843 4.5439444 4.4951425
  4.6909437]]
[[4.391132  4.407859  4.431632  4.4576735 4.437886  4.543946  4.4951444
  4.6909437]]
[[4.391128  4.4078584 4.4316344 4.4576645 4.4378867 4.543948  4.4951463
  4.6909447]]
[[4.3911195 4.407858  4.431634  4.4576526 4.4378867 4.5439496 4.495146
  4.690942 ]]
[[4.3911157 4.407853  4.431632  4.457638  4.4378853 4.543949  4.495144
  4.69094  ]]
[[4.3911195 4.407853  4.43163   4.4576507 4.4378824 4.543946  4.4951415
  4.690939 ]]
[[4.3911223 4.407852  4.431628  4.4576583 4.4378815 4.543944  4.4951396
  4.690939 ]]
[[4.391128  4.4078546 4.4316287 4.4576716 4.4378824 4.543944  4.49514
  4.69094  ]]
[[4.3911366 4.407859  4.4316306 4.457686  4.4378843 4.5439444 4.4951425
  4.6909437]]
[[4.391132  4.407859  4.431632  4.4576735 4.437886  4.543946  4.4951444
  4.6909437]]
[[4.391128  4.4078584 4.4316344 4.4576645 4.4378867 4.543948